pre processing + df encoding into list of lists (transactional data)

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

df = pd.read_csv("FormResponses1.csv")

# Removal of non standard entries
df = df[~df['Which of the tourist destinations in Baguio City will/have you visited?'].str.contains('Hindi')]

# Split the entries into lists of tourist destinations
df['Tourist Destinations'] = df['Which of the tourist destinations in Baguio City will/have you visited?'].str.split(', ')

# Dropping of the original column
df.drop(columns=['Which of the tourist destinations in Baguio City will/have you visited?'], inplace=True)

# Convert the list of tourist destinations into a list of lists
dataset = df['Tourist Destinations'].tolist()

#----end of pre processing for ARM----

# Initialization of TransactionEncoder and transformation of the dataset
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

# Create a DataFrame from the transformed array
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Display the encoded DataFrame
df_encoded.head()

#optional
#add line of code that downloads the encoded df into a csv file


,Arashiyama Bamboo Park (Baguio),Bell Church,BenCab Museum,Bencab Museum,Botanical Garden,Burnham Park,Camp John Hay,Diplomat Hotel,Ili-likha,Laperal White House,...,Our Lady of the Atonement Cathedral,PMA,SM,Session Road,Sky Ranch,Stone Kingdom,Tam-awan Village,UP Museum,Ukay ukay stores,Wright Park
0,False,False,False,False,False,True,True,False,False,False,...,False,False,True,True,False,True,False,False,False,False
1,False,True,False,False,True,True,True,True,False,False,...,True,True,True,True,True,True,True,False,True,True
2,False,False,False,False,False,True,True,True,False,False,...,False,False,False,False,False,False,True,False,False,False
3,False,True,False,False,False,True,True,False,False,False,...,True,False,True,True,True,False,False,False,True,True
4,False,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,False,False,True,True,True


Rule generation from frequent item set

initial min_support level = 0.5

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
import pandas as pd
frequent_itemsets = fpgrowth(df_encoded, min_support=0.45, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.883495,(Burnham Park)
1,0.873786,(SM)
2,0.834951,(Session Road)
3,0.679612,(Camp John Hay)
4,0.601942,(Night Market)
...,...,...
129,0.475728,"(Burnham Park, Wright Park)"
130,0.466019,"(SM, Wright Park)"
131,0.466019,"(Burnham Park, SM, Wright Park)"
132,0.456311,"(SM, Botanical Garden)"


 Block of code bellow specifies interests in rules derived from the frequent itemsets only if the level of confidence is above the 70 percent threshold (min_threshold=0.7):

In [ ]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Burnham Park),(SM),0.883495,0.873786,0.805825,0.912088,1.043834,0.033839,1.435680,0.360442
1,(SM),(Burnham Park),0.873786,0.883495,0.805825,0.922222,1.043834,0.033839,1.497920,0.332715
2,(Session Road),(SM),0.834951,0.873786,0.805825,0.965116,1.104522,0.076256,3.618123,0.573352
3,(SM),(Session Road),0.873786,0.834951,0.805825,0.922222,1.104522,0.076256,2.122053,0.749768
4,(Session Road),(Burnham Park),0.834951,0.883495,0.757282,0.906977,1.026578,0.019606,1.252427,0.156863
...,...,...,...,...,...,...,...,...,...,...
713,"(Burnham Park, Wright Park)",(SM),0.475728,0.873786,0.466019,0.979592,1.121088,0.050335,6.184466,0.206019
714,"(SM, Wright Park)",(Burnham Park),0.466019,0.883495,0.466019,1.000000,1.131868,0.054294,inf,0.218182
715,(Wright Park),"(Burnham Park, SM)",0.475728,0.805825,0.466019,0.979592,1.215638,0.082666,9.514563,0.338349
716,(Botanical Garden),(SM),0.466019,0.873786,0.456311,0.979167,1.120602,0.049109,6.058252,0.201547


If you are interested in rules according to a different metric of interest, you can simply adjust the metric and min_threshold arguments . E.g. if you are only interested in rules that have a lift score of >= 1.2, you would do the following:

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules.iloc[:, 0:7]
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(Burnham Park),(SM),0.883495,0.873786,0.805825,0.912088,1.043834
1,(SM),(Burnham Park),0.873786,0.883495,0.805825,0.922222,1.043834
2,(Session Road),(SM),0.834951,0.873786,0.805825,0.965116,1.104522
3,(SM),(Session Road),0.873786,0.834951,0.805825,0.922222,1.104522
4,(Session Road),(Burnham Park),0.834951,0.883495,0.757282,0.906977,1.026578
...,...,...,...,...,...,...,...
981,(Wright Park),"(Burnham Park, SM)",0.475728,0.805825,0.466019,0.979592,1.215638
982,(SM),(Botanical Garden),0.873786,0.466019,0.456311,0.522222,1.120602
983,(Botanical Garden),(SM),0.466019,0.873786,0.456311,0.979167,1.120602
984,(Burnham Park),(Botanical Garden),0.883495,0.466019,0.456311,0.516484,1.108288


if we are only interested in rules that satisfy the following criteria:

at least 2 antecedents
a confidence > 0.75
a lift score > 1.2
We could compute the antecedent length as follows:

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,antecedent_len
0,(Burnham Park),(SM),0.883495,0.873786,0.805825,0.912088,1.043834,1
1,(SM),(Burnham Park),0.873786,0.883495,0.805825,0.922222,1.043834,1
2,(Session Road),(SM),0.834951,0.873786,0.805825,0.965116,1.104522,1
3,(SM),(Session Road),0.873786,0.834951,0.805825,0.922222,1.104522,1
4,(Session Road),(Burnham Park),0.834951,0.883495,0.757282,0.906977,1.026578,1
...,...,...,...,...,...,...,...,...
981,(Wright Park),"(Burnham Park, SM)",0.475728,0.805825,0.466019,0.979592,1.215638,1
982,(SM),(Botanical Garden),0.873786,0.466019,0.456311,0.522222,1.120602,1
983,(Botanical Garden),(SM),0.466019,0.873786,0.456311,0.979167,1.120602,1
984,(Burnham Park),(Botanical Garden),0.883495,0.466019,0.456311,0.516484,1.108288,1


Then, we can use pandas' selection syntax as shown below:

In [ ]:
rules[ (rules['antecedent_len'] >= 1) &
       (rules['confidence'] > 0.7) &
       (rules['lift'] > 1)
       ]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,antecedent_len
0,(Burnham Park),(SM),0.883495,0.873786,0.805825,0.912088,1.043834,1
1,(SM),(Burnham Park),0.873786,0.883495,0.805825,0.922222,1.043834,1
2,(Session Road),(SM),0.834951,0.873786,0.805825,0.965116,1.104522,1
3,(SM),(Session Road),0.873786,0.834951,0.805825,0.922222,1.104522,1
4,(Session Road),(Burnham Park),0.834951,0.883495,0.757282,0.906977,1.026578,1
...,...,...,...,...,...,...,...,...
977,"(Burnham Park, Wright Park)",(SM),0.475728,0.873786,0.466019,0.979592,1.121088,2
978,"(SM, Wright Park)",(Burnham Park),0.466019,0.883495,0.466019,1.000000,1.131868,2
981,(Wright Park),"(Burnham Park, SM)",0.475728,0.805825,0.466019,0.979592,1.215638,1
983,(Botanical Garden),(SM),0.466019,0.873786,0.456311,0.979167,1.120602,1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
